<i>YT Trending Prediction</i><br>
--

Coded by :     
* Khevin Pandapotan (No GitHub)
* sammyon7

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import os
from sklearn.preprocessing import StandardScaler

Dataset <a href="https://www.kaggle.com/datasnaek/youtube-new">link</a>

In [3]:
filepath1="/kaggle/input/youtube-new/US_category_id.json"
filepath2="/kaggle/input/youtube-new/USvideos.csv"
category_id_df = pd.read_json(filepath1)
videos_df = pd.read_csv(filepath2,header='infer')

In [7]:
def clean_video_csv(video_df,country_code):
    video_df["tags"] = video_df["tags"].apply(lambda x:x.replace('"',""))
    video_df["title"] = video_df["title"].apply(lambda x:x.replace(',',' '))
    video_df["channel_title"] = video_df["channel_title"].apply(lambda x:x.replace(',',' '))
    video_df["description"] = video_df["description"].apply(lambda x:str(x).replace('\r',''))
    video_df["description"] = video_df["description"].apply(lambda x:str(x).replace(',',' '))
    video_df["description"] = video_df["description"].apply(lambda x:str(x).replace('"',''))
    video_df["country"] = country_code
    return video_df

country_code=['US']
for c in country_code:
    filepath="/kaggle/input/youtube-new/"+c+"videos.csv"
    video_df = pd.read_csv(filepath,header='infer')
    savepath = "/kaggle/working/"+c+"videos1.csv"
    video_df = clean_video_csv(video_df,c)
    video_df.to_csv(savepath,index=False)

In [9]:
def category_extract (df,country_code):
    category_id = []
    category_title = []
    for i in range(df.shape[0]):
        category_id.append(df.iloc[i]["items"]['id'])
        category_title.append(df.iloc[i]["items"]["snippet"]["title"])
    category_df = pd.DataFrame()
    category_df["category_id"] = category_id
    category_df["category_title"] = category_title
    category_df.insert(category_df.shape[1],"country_code",country_code)
    return category_df

category_all = pd.DataFrame()
for c in country_code:
    filepath="/kaggle/input/youtube-new/"+c+"_category_id.json"
    category_id_df = pd.read_json(filepath)
    category_all = pd.concat([category_all,category_extract(category_id_df,c)])
    
savepath = "/kaggle/working/category_all.csv"
category_all.to_csv(savepath,index=False)

In [10]:
US = pd.read_csv("/kaggle/working/USvideos1.csv")
category = pd.read_csv("/kaggle/working/category_all.csv")
US1 =US.merge(category,how="inner",left_on=["category_id","country"],right_on=["category_id","country_code"])
US1["trending_date1"] = US1["trending_date"].apply(lambda x: pd.Timestamp(int("20"+x[0:2]),int(x[-2:]),int(x[3:5]),0))

In [11]:
columns = ["video_id","trending_date1","channel_title","publish_time","views","likes","dislikes","comment_count","category_title"]
US1 = US1[columns].copy()

In [13]:
US1.head(10)

,video_id,trending_date1,channel_title,publish_time,views,likes,dislikes,comment_count,category_title
0,2kyS6SvSYSE,2017-11-14,CaseyNeistat,2017-11-13T17:13:01.000Z,748374,57527,2966,15954,People & Blogs
1,0mlNzVSJrT0,2017-11-14,Nobrand,2017-04-21T06:47:32.000Z,98966,2486,184,532,People & Blogs
2,STI2fI7sKMo,2017-11-14,Shawn Johnson East,2017-11-11T15:00:03.000Z,321053,4451,1772,895,People & Blogs
3,KODzih-pYlU,2017-11-14,Grace Helbig,2017-11-11T18:08:04.000Z,197062,7250,217,456,People & Blogs
4,8mhTWqWlQzU,2017-11-14,Safiya Nygaard,2017-11-11T01:19:33.000Z,2744430,115426,1110,6541,People & Blogs
5,pa_oUisZZy0,2017-11-14,BuzzFeed Celeb,2017-11-11T00:30:16.000Z,177707,6271,88,275,People & Blogs
6,fCTKDn3Q8xQ,2017-11-14,Rachel and Jun,2017-11-09T11:24:14.000Z,1098897,43875,1326,4702,People & Blogs
7,_dhneCO4YEE,2017-11-14,Full Frontal with Samantha Bee,2017-11-09T07:00:01.000Z,362009,5505,2356,1264,People & Blogs
8,EYkEshCOhEU,2017-11-14,Nicole Guerriero,2017-11-10T00:24:29.000Z,294387,15247,385,976,People & Blogs
9,Eg_kW5fw6qU,2017-11-14,SuperCarlinBrothers,2017-11-09T23:27:18.000Z,188003,9091,135,2814,People & Blogs


In [14]:
trendingdate_df = US1.groupby("video_id").trending_date1.describe(datetime_is_numeric=True).reset_index()
videos = trendingdate_df[trendingdate_df["count"].values>4].video_id
US2 = US1[ US1.video_id.isin(videos.values)]

In [15]:
print(trendingdate_df.head(10))

      video_id count                mean        min                 25%  \
0  -0CMnp02rNY     6 2018-06-08 12:00:00 2018-06-06 2018-06-07 06:00:00   
1  -0NYY8cqdiQ     1 2018-02-01 00:00:00 2018-02-01 2018-02-01 00:00:00   
2  -1Hm41N0dUs     3 2018-04-30 00:00:00 2018-04-29 2018-04-29 12:00:00   
3  -1yT-K3c6YI     4 2017-11-30 12:00:00 2017-11-29 2017-11-29 18:00:00   
4  -2RVw2_QyxQ     3 2017-11-15 00:00:00 2017-11-14 2017-11-14 12:00:00   
5  -2aVkGcI7ZA     4 2018-04-28 12:00:00 2018-04-27 2018-04-27 18:00:00   
6  -2b4qSoMnKE     2 2017-12-20 12:00:00 2017-12-20 2017-12-20 06:00:00   
7  -2wRFv-mScQ     4 2018-02-15 12:00:00 2018-02-14 2018-02-14 18:00:00   
8  -35jibKqbEo     8 2018-02-18 12:00:00 2018-02-15 2018-02-16 18:00:00   
9  -37nIo_tLnk     8 2017-12-29 12:00:00 2017-12-26 2017-12-27 18:00:00   

                  50%                 75%        max  
0 2018-06-08 12:00:00 2018-06-09 18:00:00 2018-06-11  
1 2018-02-01 00:00:00 2018-02-01 00:00:00 2018-02-01  
2 2018-04

In [18]:
def standardize(data):
    scaler = StandardScaler()
    scaler = scaler.fit(data)
    transformed = scaler.transform(data)
    return scaler,transformed

In [19]:
scaler_views, US_views = standardize(US2.views.values.reshape(-1,1))

In [24]:
US3 = pd.DataFrame()
US3["trending_date1"] = US2["trending_date1"]
US3["video_id"] = US2["video_id"]
US3["views"] = US_views
US3.reset_index(inplace=True)

In [25]:
print(US3.head(10))

   index trending_date1     video_id     views
0      0     2017-11-14  2kyS6SvSYSE -0.237504
1     17     2017-11-15  2kyS6SvSYSE -0.055430
2     31     2017-11-16  cmoknv58jjE -0.104086
3     32     2017-11-16  2kyS6SvSYSE -0.038156
4     33     2017-11-16  Jidk0O6uu-0 -0.325464
5     42     2017-11-17  t1shnJT8NCY -0.281801
6     44     2017-11-17  cmoknv58jjE -0.062990
7     46     2017-11-17  Jidk0O6uu-0 -0.323873
8     47     2017-11-17  2kyS6SvSYSE -0.028610
9     54     2017-11-18  t1shnJT8NCY -0.265967


In [26]:
US3.drop("index",axis=1,inplace=True)
US3.head(10)

,trending_date1,video_id,views
0,2017-11-14,2kyS6SvSYSE,-0.237504
1,2017-11-15,2kyS6SvSYSE,-0.055430
2,2017-11-16,cmoknv58jjE,-0.104086
3,2017-11-16,2kyS6SvSYSE,-0.038156
4,2017-11-16,Jidk0O6uu-0,-0.325464
5,2017-11-17,t1shnJT8NCY,-0.281801
6,2017-11-17,cmoknv58jjE,-0.062990
7,2017-11-17,Jidk0O6uu-0,-0.323873
8,2017-11-17,2kyS6SvSYSE,-0.028610
9,2017-11-18,t1shnJT8NCY,-0.265967


In [27]:
x=[]
y=[]
category = []
for v in videos:
    row=[]
    temp_df = US3[US3["video_id"]==v].sort_values(by="trending_date1")
    #print (temp_df)
    seq = temp_df.views[0:4].index
        
    for s in seq:
        row.append(US3.iloc[s].values[2:3])
    x.append(row)
    nextstep = temp_df.views[4:5].values
    y.append(nextstep)

In [28]:
x = np.reshape(x,(len(x),4,1))
print (x.shape)
print (x[0])

(3984, 4, 1)
[[-0.2719427408449598]
 [-0.2555659799132111]
 [-0.24286315271238587]
 [-0.23675864020197335]]


In [29]:
y = np.reshape(y,(-1,1))
print (y.shape)
print (y[0])

(3984, 1)
[-0.23191164]


In [30]:
x = x.astype('float32')
y = y.astype('float32')

# Train,validation and test data split. Each one should be integral multiples of batchsize.
batch_size = 100

x_train,x_remain = x[:3700],x[3700:3900]
y_train,y_remain = y[:3700],y[3700:3900]

x_val,x_test = x_remain[:100],x_remain[100:]
y_val,y_test = y_remain[:100],y_remain[100:]
print (x_train.shape,x_val.shape,x_test.shape)

(3700, 4, 1) (100, 4, 1) (100, 4, 1)


In [31]:
import torch
from torch.utils.data import TensorDataset,DataLoader

In [35]:
train_data = TensorDataset(torch.from_numpy(x_train),torch.from_numpy(y_train))
valid_data = TensorDataset(torch.from_numpy(x_val),torch.from_numpy(y_val))
test_data = TensorDataset(torch.from_numpy(x_test),torch.from_numpy(y_test))


train_loader = DataLoader(train_data,shuffle=True,batch_size=batch_size)
valid_loader = DataLoader(valid_data,shuffle=True,batch_size=batch_size)
test_loader = DataLoader(test_data,shuffle=True,batch_size=batch_size)

In [36]:
dataiter = iter(train_loader)
sample_x,sample_y = dataiter.next()

print ('Sample input size:',sample_x.size())
print ('Sample input:\n',sample_x[0:2])
print ('\n')
print ('Sample output size:',sample_y.size())
print ('Sample output:\n',sample_y[0:2])

Sample input size: torch.Size([100, 4, 1])
Sample input:
 tensor([[[-0.2593],
         [-0.2489],
         [-0.2456],
         [-0.2437]],

        [[-0.3068],
         [-0.3050],
         [-0.3040],
         [-0.3034]]])


Sample output size: torch.Size([100, 1])
Sample output:
 tensor([[-0.2418],
        [-0.3027]])


<i>Now create the training model!</i>

In [40]:
import torch.nn as nn

class mylstm(nn.Module):
    
    def __init__(self,input_size,output_size,hidden_dim, n_layers,drop_prob=0.5):
        super(mylstm,self).__init__()
        self.input_size = input_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        self.lstm = nn.LSTM(input_size,hidden_dim,n_layers,batch_first=True)
        self.dropout = nn.Dropout(0.3)
        self.fc = nn.Linear(hidden_dim,output_size)
        
    def forward(self,x,hidden):
        batch_size = x.size(0)
        
        lstm_out,hidden = self.lstm(x,hidden)
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
        out =self.dropout(lstm_out)
       
        out = self.fc(out)#shape=(batchsize,seqlen,outputdim)
        out = out.view(batch_size, -1) 
        out = out[:, -1]
        return out,hidden
    
    def init_hidden(self,batch_size):
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        
        return hidden

In [41]:
input_size = 1
output_size =1
hidden_dim = 256
n_layers = 2

net = mylstm(input_size,output_size,hidden_dim,n_layers)
print (net)

mylstm(
  (lstm): LSTM(1, 256, num_layers=2, batch_first=True)
  (dropout): Dropout(p=0.3, inplace=False)
  (fc): Linear(in_features=256, out_features=1, bias=True)
)


In [43]:
train_on_gpu = torch.cuda.is_available()
if train_on_gpu:
    print ('Training on GPU')
else:
    print ('Training on CPU')

Training on CPU


In [45]:
# Now train!
lr = 0.001
epoch = 40
counter = 0
print_every = 50
clip = 5

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(net.parameters(),lr=lr)

if train_on_gpu:
    net.cuda()
    
net.train()
for e in range(epoch):
    h = net.init_hidden(batch_size)
    for inputs,labels in train_loader:
        counter +=1
        if train_on_gpu:
            inputs,labels = inputs.cuda(),output.cuda()
            
        h = tuple([each.data for each in h])
        net.zero_grad()
        output,h = net(inputs,h)
        
        loss = criterion(output.squeeze(),labels.squeeze())
        loss.backward()
        
        nn.utils.clip_grad_norm_(net.parameters(),clip)
        optimizer.step()
        
        if counter % print_every==0:
            val_h = net.init_hidden(batch_size)
            val_losses = []
            net.eval()
            for inputs,labels in valid_loader:
                val_h = tuple([each.data for each in val_h])
                if train_on_gpu:
                    inputs,labels = inputs.cuda(),labels.cuda()
                output,val_h = net(inputs,val_h)
                val_loss = criterion(output.squeeze(),labels.squeeze())
                
                val_losses.append(val_loss.item())
            net.train()
            print ("Epoch:{}/{}...".format(e+1,epoch),
               "Step:{}...".format(counter),
               "Loss:{:.6f}...".format(loss.item()),
               "Val loss:{:.6f}".format(np.mean(val_losses)))

Epoch:2/40... Step:50... Loss:0.059253... Val loss:0.009478
Epoch:3/40... Step:100... Loss:0.114847... Val loss:0.069680
Epoch:5/40... Step:150... Loss:0.005290... Val loss:0.003051
Epoch:6/40... Step:200... Loss:0.002424... Val loss:0.007196
Epoch:7/40... Step:250... Loss:0.003964... Val loss:0.001407
Epoch:9/40... Step:300... Loss:0.004749... Val loss:0.003351
Epoch:10/40... Step:350... Loss:0.001159... Val loss:0.001201
Epoch:11/40... Step:400... Loss:0.000471... Val loss:0.000984
Epoch:13/40... Step:450... Loss:0.002015... Val loss:0.001076
Epoch:14/40... Step:500... Loss:0.010099... Val loss:0.009626
Epoch:15/40... Step:550... Loss:0.000590... Val loss:0.000712
Epoch:17/40... Step:600... Loss:0.000682... Val loss:0.001411
Epoch:18/40... Step:650... Loss:0.002139... Val loss:0.000706
Epoch:19/40... Step:700... Loss:0.000856... Val loss:0.001039
Epoch:21/40... Step:750... Loss:0.004292... Val loss:0.002492
Epoch:22/40... Step:800... Loss:0.000676... Val loss:0.000441
Epoch:23/40... 

In [47]:
test_losses=[]

h = net.init_hidden(batch_size)
net.eval()
for inputs,labels in test_loader:
    h = tuple([each.data for each in h])
    if train_on_gpu:
        inputs,labels =inputs.cuda(),labels.cuda()
    output,h = net(inputs,h)
    test_loss = criterion(output.squeeze(),labels.squeeze())
    
    test_losses.append(test_loss.item())

print ("Test loss is : {:.6f}".format(np.mean(test_losses)))

Test loss is : 0.000831


In [50]:
from torch import Tensor 
from sklearn.metrics import mean_squared_error
import math
predict_tensor = torch.from_numpy(x[3900:3910])
batch_size = predict_tensor.size(0)
h = net.init_hidden(batch_size)
if train_on_gpu:
    predict_tensor = predict_tensor.cuda()
output,h = net(predict_tensor,h)
pred = scaler_views.inverse_transform(Tensor.detach(output) )
true = scaler_views.inverse_transform(y[3900:3910])
for i in range(10):
    print ("The true value is {}, and the predict value is {}".format(true[i],pred[i]))
print ("The RMSE : ",math.sqrt(mean_squared_error(pred,true)))

The true value is [3017144.], and the predict value is 3154952.0
The true value is [237230.97], and the predict value is 246670.46875
The true value is [460125.97], and the predict value is 472240.96875
The true value is [1899969.], and the predict value is 1961960.375
The true value is [879709.94], and the predict value is 867860.3125
The true value is [1827806.], and the predict value is 1873080.375
The true value is [96253.21], and the predict value is 97961.7109375
The true value is [351798.97], and the predict value is 370323.46875
The true value is [99200.96], and the predict value is 98440.7109375
The true value is [608431.06], and the predict value is 622209.9375
The RMSE :  50790.3033265209


In [52]:
print ("The average percentage error 10 data is {:.3f}%".format(np.mean((pred-true)/true)))

The average percentage error 10 data is 2.309%
